In [11]:
import pandas as pd
import requests

In [3]:
from src.data.cloud._constants import Cloud

In [20]:
for c in [c.value.lower() for c in Cloud]:
    df = pd.read_csv(f'../data/raw/{c}_services.csv')
    df = df[['category_name', 'name', 'short_description', 'long_description']]
    norm_cats = {}
    for cat in df['category_name'].unique():
        norm_cats[cat] = get_normalized_cat_name(c, cat)
    df['category_name'] = df['category_name'].apply(lambda cat_name: norm_cats[cat_name])
    df.to_csv(f'../data/processed/{c}_dedupe_data.csv', index=False)

alibaba
aws
azure
digitalocean
gcp
ibm
oracle


In [15]:
def get_normalized_cat_name(abbr, cat_name):
    q = {
        "query": f"g.V().has('abbreviation', '{abbr}').in('source_cloud').has('name', '{cat_name}').out('super_category').values('name')"
    }
    
    res = requests.post('https://cloudconceptgraph.azure-api.net/gremlin', json=q)
    data = res.json()
    if len(data) == 1:
        return data[0]
    else:
        return cat_name
get_normalized_cat_name('ibm', 'Network')

'Networking & Content Delivery'